In [1]:
#!/usr/bin/python3
import sys
import os
import torch
from peft import PeftModel, set_peft_model_state_dict
from transformers import AutoTokenizer, AutoModelForCausalLM
from prompt import make_prompt

BASE_MODEL = "PygmalionAI/pygmalion-6b"
PEFT_WEIGHTS = "out/Curio_v1/"

# BASE_MODEL = "EleutherAI/gpt-j-6B"
# PEFT_WEIGHTS = "Curio/checkpoint-600/"
load_in_8bit = True


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/dungnt/anaconda3/envs/chai-dont/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA SETUP: CUDA runtime path found: /home/dungnt/anaconda3/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.9
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /home/dungnt/anaconda3/envs/chai-dont/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/home/dungnt/anaconda3/envs/chai-dont/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/home/dungnt/anaconda3/lib/libcudart.so'), PosixPath('/home/dungnt/anaconda3/lib/libcudart.so.11.0')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


In [2]:
# model = AutoModelForCausalLM.from_pretrained(BASE_MODEL)
model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL,
        load_in_8bit=load_in_8bit,
        torch_dtype=torch.float16,
        device_map="auto",
    )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
model = PeftModel.from_pretrained(model, PEFT_WEIGHTS)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
model.eval()
print("Let's gooo")

Let's gooo


In [4]:
def get_answer(prompt, max_new_tokens=1024, skip_tl=False):
    input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"].to(model.device)
    with torch.no_grad():
        gen_tokens = model.generate(
            input_ids=input_ids,
            max_length=len(input_ids) + max_new_tokens,
            do_sample=True,
            temperature=0.5,
            top_k=20,
            repetition_penalty=1.2,
            # eos_token_id=0, # for open-end generation.
            pad_token_id=tokenizer.eos_token_id,
        )
    origin_output = tokenizer.batch_decode(gen_tokens)[0]
    output = origin_output.split("I'd like to ask:")[-1]
    try:
        k = output.index(":")
        if k < 10: output = output[k+1:]
    except:
        output = output
    # print(f"\n- - -{origin_output}- - -\n")
    output = output.split("<|endoftext|>")[0]
    return output.strip()

In [5]:
from prompt import make_curious_prompt
title = "Walter Sickert"
context = """
In 1926 he suffered an illness, thought to have been a minor stroke.[22] In 1927, he abandoned his first name in favour of his middle name, and thereafter chose to be known as Richard Sickert.[23] His style and subject matter also changed: Sickert stopped drawing, and instead painted from snapshots usually taken by his third wife, Thérèse Lessore, or from news photographs. The photographs were squared up for enlargement and transferred to canvas, with their pencil grids plainly visible in the finished paintings.
"""
prompt = make_curious_prompt(title, context, "") + "1. What"
print(prompt)

Curio's Persona: Curio is a person who has a strong desire to learn and understand things better. Whenever he reads something, he likes to ask questions to satisfy his curiosity. 
<START>
You: Make plausible questions about the following context.
### TITLE: Walter Sickert
### CONTENT: 
In 1926 he suffered an illness, thought to have been a minor stroke.[22] In 1927, he abandoned his first name in favour of his middle name, and thereafter chose to be known as Richard Sickert.[23] His style and subject matter also changed: Sickert stopped drawing, and instead painted from snapshots usually taken by his third wife, Thérèse Lessore, or from news photographs. The photographs were squared up for enlargement and transferred to canvas, with their pencil grids plainly visible in the finished paintings.

Curio: Here are some questions I'd like to ask:
1. What


In [6]:
model.device

device(type='cuda', index=0)

In [8]:
%%time
print(get_answer(prompt))

1. What did Walter sicker think was wrong?
2. Who took most pictures of Walter sicker?
CPU times: user 19.1 s, sys: 460 µs, total: 19.1 s
Wall time: 19.1 s
